In [1]:
import numpy as np 
from scipy.stats import unitary_group
import strawberryfields as sf 
import strawberryfields.ops as ops
from chain_rule import get_samples, get_samples_click
from thewalrus.quantum import photon_number_mean_vector, mean_clicks
from MIS import MIS_IPS
from MIS_click import ClickMIS

## make a gaussian covariance matrix for a GBS experiment

In [2]:
M = 4
eng = sf.Engine(backend='gaussian')
prog = sf.Program(M)
U = unitary_group.rvs(M)

r = 1
eta = 0.9
alpha = 0.1
with prog.context as q:
    for i in range(M):
        ops.Sgate(r) | q[i]
        ops.LossChannel(eta) | q[i]
    ops.Interferometer(U) | q
    
state = eng.run(prog).state

# get wigner function displacement and covariance
mu = state.means()
cov = state.cov()

In [3]:
# chain rule sampling with PNRDs

for sample in get_samples(mu, cov, n_samples=10):
    print(sample)

[4 5 0 6]
[9 7 1 0]
[1 2 0 1]
[0 0 3 0]
[8 7 4 1]
[1 1 3 4]
[5 1 1 5]
[5 0 5 4]
[0 1 0 0]
[0 0 1 1]


In [4]:
# chain rule sampling with threshold detectors

for sample in get_samples_click(mu, cov, n_samples=10):
    print(sample)

[0 0 1 1]
[1 1 0 0]
[1 1 1 1]
[0 0 0 0]
[1 1 1 1]
[0 0 0 0]
[1 1 1 1]
[1 1 0 0]
[1 1 0 0]
[1 1 0 1]


In [5]:
# MIS with PNRDs
N = int(np.round(photon_number_mean_vector(mu, cov)).sum())
mis = MIS_IPS(cov, N)
mis.run_chain(120)

burn_in = 20 
thinning_rate = 10

print(np.array(mis.chain_patterns[burn_in::thinning_rate]))

[[0 0 2 2]
 [1 1 1 1]
 [0 2 0 2]
 [0 1 0 3]
 [2 2 0 0]
 [2 2 0 0]
 [0 2 0 2]
 [2 0 2 0]
 [2 2 0 0]
 [0 1 2 1]
 [0 0 3 1]]


In [6]:
# MIS with threshold detectors
N = int(np.round(mean_clicks(cov)))
mis = ClickMIS(cov, N)
mis.run_chain(120)

burn_in = 20 
thinning_rate = 10

print(np.array(mis.chain_patterns[burn_in::thinning_rate]))

[[1 1 0 0]
 [0 0 1 1]
 [0 0 1 1]
 [0 1 1 0]
 [0 1 0 1]
 [1 1 0 0]
 [1 1 0 0]
 [1 1 0 0]
 [1 1 0 0]
 [1 0 1 0]
 [0 1 0 1]]
